In [ ]:
# Modules and settings
import importlib
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import numpy as np
import rasterio
import sys
import os
import warnings
import xarray as xr

from pysheds.grid import Grid
sys.path.append("..")
from models import STREAM_model as STREAM 
# Hide warnings regarding nans in coditional statements 
np.seterr(invalid='ignore')
warnings.filterwarnings('ignore')

#%matplotlib inline
%autosave 60

In [ ]:
# Parameters
dischargepointrow = 173
dischargepointcolumn = 196
pixel_area_m2 = 2118000 
timestep = 24*3600*30

In [ ]:
# Start and end time of iteration
timestart =108   # Start month of simulation 
timeend = 168        # End month of simulation

Create list of months and years of the full extend of the data. Open *year_list* and *month_list* in the Variable Inspector (right-click to open) after running the next cell to check what is in these lists (i.e. what is calculated in the next cell).

In [ ]:
start_year = 1901
end_year = 2023
start_month = 0    # January 
end_month = 12     #December 

month_list, year_list = STREAM.set_dates(start_year, end_year, start_month, end_month)

In [ ]:

# Define the input directory path once
input_dir = os.path.join('..', 'models', 'Input_file')

# read Land characteristics files
C      = rasterio.open(os.path.join(input_dir, 'slope_class_percentiles.tif')).read()[0] # Slope class map
Cropf  = rasterio.open(os.path.join(input_dir, 'CropFactor_v5.tif')).read()[0]  # Crop Factor map
#Cropf = Cropf / 10 #Adjust Crop factor values if necessary
Wholdn = rasterio.open(os.path.join(input_dir, 'WHC.tif')).read()[0]    # Water holding capacity map
Wholdn[Wholdn == -9999] = 5 
DEM    = rasterio.open(os.path.join(input_dir, 'DEM.tif')).read()[0]        # Digital Elevation Model
# read initial storage files
Aw     = np.ones_like(DEM)
Gw     = np.ones_like(DEM)
Snow   = np.ones_like(DEM)

Temperature   = xr.open_dataset(os.path.join(input_dir, 'temp_subset.dat.nc')).to_array().values[0]
A             = xr.open_dataset(os.path.join(input_dir, 'A_resampled.nc')).to_array().values[0]
Precipitation = xr.open_dataset(os.path.join(input_dir, 'precipitation_subset.dat.nc')).to_array().values[0]
Heat          = xr.open_dataset(os.path.join(input_dir, 'HEAT_resampled.nc')).to_array().values[0]
Heat[Heat == 0] = 32.0  # Adjust heat values of zero to a mean of 32

In [ ]:
print("A map:", A.shape)
print("Heat map:", Heat.shape)

In [ ]:
def visualise_input_data(visualise=True):
    if visualise == True:

        # Display multiple maps side by side
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle("Visualised input data", fontsize=24, y=1.02)

        # 1. Slope Class (Discrete)
        classes = [1, 2, 3, 4]
        cmap_slope = colors.ListedColormap(['#440154', '#31688e', '#35b779', '#fde725'])
        bounds = [0.5, 1.5, 2.5, 3.5, 4.5]
        norm = colors.BoundaryNorm(bounds, cmap_slope.N)

        im0 = axes[0, 0].imshow(C, cmap=cmap_slope, norm=norm)
        axes[0, 0].set_title('Slope Class')
        cbar0 = fig.colorbar(im0, ax=axes[0, 0], ticks=classes)
        cbar0.ax.set_yticklabels(['1', '2', '3', '4'])
        cbar0.set_label('Slope Class', rotation=270, labelpad=15)

        # 2. Crop Factor
        im1 = axes[0, 1].imshow(Cropf, cmap='YlGn')
        axes[0, 1].set_title('Crop Factor')
        cbar1 = fig.colorbar(im1, ax=axes[0, 1])
        cbar1.set_label('Crop Factor', rotation=270, labelpad=15)

        # 3. Water Holding Capacity
        im2 = axes[1, 0].imshow(Wholdn, cmap='Blues')
        axes[1, 0].set_title('Water Holding Capacity')
        cbar2 = fig.colorbar(im2, ax=axes[1, 0])
        cbar2.set_label('WHC (mm)', rotation=270, labelpad=15)

        # 4. DEM
        im3 = axes[1, 1].imshow(DEM, cmap='terrain')
        axes[1, 1].set_title('DEM')
        cbar3 = fig.colorbar(im3, ax=axes[1, 1])
        cbar3.set_label('Elevation (m)', rotation=270, labelpad=15)
    
        plt.tight_layout()
        plt.show()

# Set the parameter "visualise_input_data" to True to visualise the input data maps
visualise_input_data(visualise=False)

In [ ]:
input_dir = os.path.join('..', 'models', 'Input_file')
dem_path = os.path.join(input_dir, 'DEM.tif')

# Use it for all file reads
grid = Grid.from_raster(dem_path)
dem_grid = grid.read_raster(dem_path)

# Fill pits in DEM
pit_filled_dem = grid.fill_pits(dem_grid)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(pit_filled_dem)
    
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)

# Compute flow directions
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

# Load the same DEM for STREAM model
DEM = rasterio.open(dem_path).read()[0]


In [ ]:
#setup the model and pass all the relevant information to the model setup function. 
importlib.reload(STREAM)

elbe = STREAM.Setup_model(dischargepointrow, dischargepointcolumn, pixel_area_m2,
                              timestep, C, Cropf, Wholdn, Aw, Gw, Snow,
                              Precipitation, Temperature, Heat, A, grid, fdir, dirmap,
                              month_list, year_list)

Calibrating the model
We use the following variables in our model: \
*GW*     = Groundwater \
*Tmp*    = Temperature \
*Pe*     = Potential evapotranspiration \
*Pre*    = Precipitation \
*Excess* = Excess water, or surface runoff after evaporation and saturation of the soil \
*Aw*     = Available water (for plants) in the soil   \
*Runoff* = Runoff (both surface and subsurface) that leaves the grid cell. This is not accumulated/routed yet. \
*Qsec*   = routed or accumulated discharge in cubic meters per second. \

In [ ]:
elbe.dischargepointrow -= 1
elbe.dischargepointcolumn -= 1

In [ ]:
elbe.model_flow(timestart=timestart, timeend=timeend)

TimeseriesGw     = elbe.TimeseriesGw['Gw']
TimeseriesTmp    = elbe.TimeseriesTmp['Tmp']
TimeseriesPe     = elbe.TimeseriesPe['Pe']
TimeseriesPre    = elbe.TimeseriesPre['Pre']
TimeseriesExcess = elbe.TimeseriesExcess['Excess']
TimeseriesAw     = elbe.TimeseriesAw['Aw']
TimeseriesRunoff = elbe.TimeseriesRunoff['Runoff']
TimeseriesQsec   = elbe.TimeseriesQsec['Qsec']

In [ ]:
list_of_runs = [TimeseriesQsec]
list_of_labels = ['Current climate']
yearly_average_comparison(start_year, timestart, timeend, list_of_runs, list_of_labels)

In [ ]:
# 1. Check if the model ran at all
print(f"Timestart: {timestart}, Timeend: {timeend}")
print(f"Number of timesteps: {timeend - timestart}")

# 2. Check what timeseries actually have data
print("\nTimeseries lengths:")
print(f"Gw: {len(TimeseriesGw)}")
print(f"Tmp: {len(TimeseriesTmp)}")
print(f"Pre: {len(TimeseriesPre)}")
print(f"Pe: {len(TimeseriesPe)}")
print(f"Excess: {len(TimeseriesExcess)}")
print(f"Aw: {len(TimeseriesAw)}")
print(f"Runoff: {len(TimeseriesRunoff)}")
print(f"Qsec: {len(TimeseriesQsec)}")

# 3. Check if discharge point is valid
print(f"\nDischarge point: row={dischargepointrow}, col={dischargepointcolumn}")
print(f"Grid shape: {grid.shape}")
print(f"Is discharge point in grid? {0 <= dischargepointrow < grid.shape[0] and 0 <= dischargepointcolumn < grid.shape[1]}")

# # 4. Check input data for NaNs
# print("\nChecking for NaN values:")
# print(f"Precipitation NaNs: {Precipitation.isnan().sum().values}")
# print(f"Temperature NaNs: {Temperature.isnan().sum().values}")

# # 5. Look at intermediate values during a single timestep
# # Try running just one timestep manually if possible
# print("\nInitial conditions:")
# print(f"Aw initial: {Aw[0, dischargepointrow, dischargepointcolumn]}")
# print(f"Gw initial: {Gw[0, dischargepointrow, dischargepointcolumn]}")

In [ ]:
# #patterns we can detect for each variable 
# %matplotlib inline
# importlib.reload(STREAM)
# elbe.plot_series()

In [ ]:
# # Adjust Start time of iteration below
# timestart = 108 - 12     # Start month of calibration
# timeend   = 168      # End month of calibration

# elbe.model_flow(timestart=timestart, timeend=timeend)
# qstation = elbe.TimeseriesQsec
# qstation.to_excel('calibration.xlsx')

In [ ]:
# # Adjusting Calibration parameters
# elbe.HEATcal   = 1.0     # Not used in this practical: default=1 [>0]
# elbe.WHOLDNcal = 1     # Water holding capacity of the soil: default=1 [>0]
# elbe.MELTcal   = 10   # How fast snow melts: default=10 [>0]
# elbe.CROPFcal  = 1     # Parameter steering the evapotranspiration: default=1 [>0]
# elbe.TOGWcal   = 0.4     # Parameter seperating the fraction going to groundwater and direct runoff: default=0.4 [0-1]
# elbe.Ccal      = 1     # Parameter steering how fast groundwater flows: default=1 [>1]

# elbe.model_flow(timestart=timestart, timeend=timeend)
# qstation = elbe.TimeseriesQsec
# qstation.to_excel('adjusting_calibration_parameters.xlsx')

In [ ]:
# Completing a long run --> to see how the model performs over an extended period

# timestart = 1        # Start month of simulation
# timeend   = 1128        # End month of simulation
# elbe.model_flow(timestart=timestart, timeend=timeend)
# qstation = elbe.TimeseriesQsec
# qstation.to_excel('longrun.xlsx')

In [ ]:
#inspect the temporal and spatial patterns of the variables 
# elbe.animate()